# The Diet Problem

The Diet Problem was motivated by the Army's desire to meet the nutritional requirements of its soldiers while minimizing the cost. Here, we solve the diet problem with real data. We investigate two scenarios. First we investigate the basic meal plan where we only focus on the nutrion and cost. Laterr, we consider taste and flavour as part of the optimization problem.

## Problem two: Premium Meals
Here we, formulate the optimization model as a linear program, to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, but consider the following constratints as well:

  + a.	If a food is selected, then a minimum of 1/10 serving must be chosen.
  
  + b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
  
  + c.	At least 3 kinds of meat/poultry/fish/eggs must be selected.


In [1]:
from pulp import *
import pandas as pd

# 1. Read and explore the data

Here, we load the data from excel sheet, and explore the data

In [2]:
# load the diet data
df = pd.read_excel( open('data/diet.xls','rb'),sheet_name='Sheet1')

df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


# 2. Clean the Data

We just explore the data, and make sure the format and values are correct. Here, we noticed that the last 3 rows of xls file contains one empty row, minimum daily intake, and maximum daily intake

In [3]:
df.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
66,NaN,NaN,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


In [4]:
# data cleaning
data = df[0:64].copy()
data.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0


# 3. Data Prepration

Here, we orgonize the data into more readible formats. 

+ We create some dictionaries for each listed food item and its corresponding nutrition value.
+ We create two dictionaries for Minimum and Maximum daily intake
+ We create a list of name of the foods and nutritions

In [5]:
foods_dict = data.set_index('Foods').to_dict()
# Sample output
print("\n\nCalcium mg\n", foods_dict['Calcium mg'])



Calcium mg
 {'Frozen Broccoli': 159.0, 'Carrots,Raw': 14.9, 'Celery, Raw': 16.0, 'Frozen Corn': 3.3, 'Lettuce,Iceberg,Raw': 3.8, 'Peppers, Sweet, Raw': 6.7, 'Potatoes, Baked': 22.7, 'Tofu': 121.8, 'Roasted Chicken': 21.9, 'Spaghetti W/ Sauce': 80.2, 'Tomato,Red,Ripe,Raw': 6.2, 'Apple,Raw,W/Skin': 9.7, 'Banana': 6.8, 'Grapes': 3.4, 'Kiwifruit,Raw,Fresh': 19.8, 'Oranges': 52.4, 'Bagels': 21.0, 'Wheat Bread': 10.8, 'White Bread': 26.2, 'Oatmeal Cookies': 6.7, 'Apple Pie': 3.1, 'Chocolate Chip Cookies': 6.2, 'Butter,Regular': 1.2, 'Cheddar Cheese': 202.0, '3.3% Fat,Whole Milk': 291.3, '2% Lowfat Milk': 296.7, 'Skim Milk': 302.3, 'Poached Eggs': 24.5, 'Scrambled Eggs': 42.6, 'Bologna,Turkey': 23.8, 'Frankfurter, Beef': 9.0, 'Ham,Sliced,Extralean': 2.0, 'Kielbasa,Prk': 11.4, "Cap'N Crunch": 4.8, 'Cheerios': 48.6, "Corn Flks, Kellogg'S": 0.9, "Raisin Brn, Kellg'S": 12.9, 'Rice Krispies': 4.0, 'Special K': 8.2, 'Oatmeal': 18.7, 'Malt-O-Meal,Choc': 23.1, 'Pizza W/Pepperoni': 64.6, 'Taco': 220

In [6]:
# Minimum daily intake
dmin = df.iloc[65,3:].to_dict()
# Maximum daily intake
dmax = df.iloc[66,3:].to_dict()

print("Minimum daily intake:\n", dmin)

print("\n\n Maximum daily intake:\n", dmax)

Minimum daily intake:
 {'Calories': 1500.0, 'Cholesterol mg': 30.0, 'Total_Fat g': 20.0, 'Sodium mg': 800.0, 'Carbohydrates g': 130.0, 'Dietary_Fiber g': 125.0, 'Protein g': 60.0, 'Vit_A IU': 1000.0, 'Vit_C IU': 400.0, 'Calcium mg': 700.0, 'Iron mg': 10.0}


 Maximum daily intake:
 {'Calories': 2500.0, 'Cholesterol mg': 240.0, 'Total_Fat g': 70.0, 'Sodium mg': 2000.0, 'Carbohydrates g': 450.0, 'Dietary_Fiber g': 250.0, 'Protein g': 100.0, 'Vit_A IU': 10000.0, 'Vit_C IU': 5000.0, 'Calcium mg': 1500.0, 'Iron mg': 40.0}


In [7]:
foods = list(data['Foods'])
constraints = list(foods_dict.keys())[3:]
protein_index = [8,27,28,29,30,31,32,41,42,43,44,49,50,51,56,57,59,61,63]
protein = [foods[i] for i in protein_index]


print("foods:\n", foods)

print("\n\nprotein:\n", protein)

print("\n\nconstraints:\n", constraints)


foods:
 ['Frozen Broccoli', 'Carrots,Raw', 'Celery, Raw', 'Frozen Corn', 'Lettuce,Iceberg,Raw', 'Peppers, Sweet, Raw', 'Potatoes, Baked', 'Tofu', 'Roasted Chicken', 'Spaghetti W/ Sauce', 'Tomato,Red,Ripe,Raw', 'Apple,Raw,W/Skin', 'Banana', 'Grapes', 'Kiwifruit,Raw,Fresh', 'Oranges', 'Bagels', 'Wheat Bread', 'White Bread', 'Oatmeal Cookies', 'Apple Pie', 'Chocolate Chip Cookies', 'Butter,Regular', 'Cheddar Cheese', '3.3% Fat,Whole Milk', '2% Lowfat Milk', 'Skim Milk', 'Poached Eggs', 'Scrambled Eggs', 'Bologna,Turkey', 'Frankfurter, Beef', 'Ham,Sliced,Extralean', 'Kielbasa,Prk', "Cap'N Crunch", 'Cheerios', "Corn Flks, Kellogg'S", "Raisin Brn, Kellg'S", 'Rice Krispies', 'Special K', 'Oatmeal', 'Malt-O-Meal,Choc', 'Pizza W/Pepperoni', 'Taco', 'Hamburger W/Toppings', 'Hotdog, Plain', 'Couscous', 'White Rice', 'Macaroni,Ckd', 'Peanut Butter', 'Pork', 'Sardines in Oil', 'White Tuna in Water', 'Popcorn,Air-Popped', 'Potato Chips,Bbqflvr', 'Pretzels', 'Tortilla Chip', 'Chicknoodl Soup', 'Splt 

# 4. Define the optimization problem framework

To define the optimization problem, we need to define the variables, constraines and objective funtions. 

In [8]:
# create the minimization problem framework 
prob2 = LpProblem('PuLPTutorial', LpMinimize)

In [9]:
# Define a dictionary for foods referenced Variables (# of serving)
food_vars = LpVariable.dicts('Food', foods, lowBound=0, cat='Continuous')

In [10]:
# Define a dictionary for foods -  (chosen or not)
chosen_Vars = LpVariable.dicts("Chosen", foods, lowBound=0, upBound=1, cat="Binary")

In [11]:
# define the objective function: Cost of Foods per person
prob2 += lpSum([foods_dict['Price/ Serving'][f] * food_vars[f] for f in foods])

In [12]:
# Add maximum and minimum daily nutrition constraints
for constraint in constraints:
    prob2 += lpSum([foods_dict[constraint][f] * food_vars[f] for f in foods]) >= dmin[constraint], constraint + "minRequirement"
    prob2 += lpSum([foods_dict[constraint][f] * food_vars[f] for f in foods]) <= dmax[constraint], constraint + "MaxRequirement"
    

### Add new constraints defined in problem 2

In [13]:
# Add constraints for : if any, minimum of 1/10 serving but less than maximum value
for f in foods:
    prob2 += chosen_Vars[f] * 0.1 <= food_vars[f]
    prob2 += chosen_Vars[f] * 10000000000 >= food_vars[f]


# The following two lines did not work. Because lpSum just sumes the expressions not conditions
#prob2 += lpSum([chosen_Vars[f] * 0.1 <= food_vars[f] for f in foods]) 
#prob2 += lpSum([chosen_Vars[f] * 10000000000 >= food_vars[f] for f in foods])


In [14]:
# Add constraints for : at most one, but not both celery and frozen broccoli.
prob2 += chosen_Vars['Frozen Broccoli'] + chosen_Vars['Celery, Raw'] <=1, "celerybroccoliConstraints"


In [15]:
# Add constraints for : at least 3 kinds of meat/poultry/fish/eggs
prob2 += lpSum([chosen_Vars[p] for p in protein]) >= 3, "proteinConstraints"


In [16]:
# Write down the the problem data
#prob2.writeLP("DietModelforProblem2.lp")

# 5. Solve the optimization and print the calulated values

Here, we solve the optimization problem, and then print the calculated **Non-Zero** values.

In [17]:
# Solve the problem
prob2.solve()

# Print the solution
print ("Status:", LpStatus[prob2.status])

Status: Optimal


In [18]:
# Print the calculated optimum serving for each food
for serving in prob2.variables():
    if serving.varValue > 0:
        print (serving.name, " : ", serving.varValue)

Chosen_Celery,_Raw  :  1
Chosen_Kielbasa,Prk  :  1
Chosen_Lettuce,Iceberg,Raw  :  1
Chosen_Oranges  :  1
Chosen_Peanut_Butter  :  1
Chosen_Poached_Eggs  :  1
Chosen_Popcorn,Air_Popped  :  1
Chosen_Scrambled_Eggs  :  1
Food_Celery,_Raw  :  38.6326
Food_Kielbasa,Prk  :  0.1
Food_Lettuce,Iceberg,Raw  :  86.9607
Food_Oranges  :  3.18078
Food_Peanut_Butter  :  2.73881
Food_Poached_Eggs  :  0.1
Food_Popcorn,Air_Popped  :  13.083
Food_Scrambled_Eggs  :  0.1


In [19]:
# Print the minimum Cost Per Person    
print ("\n\nCost of Food per person = ", value(prob2.objective))



Cost of Food per person =  4.5106717


# Analysis of the results:

Let's review the results:

+ The daily cost per person for basic choise/part1: 4.33711564

+ The daily cost per person for tasty choice/part2: 4.5106717

So, one may think it worth it to update the meal-plan from basic to tasty for just ~ $0.2 per person per day, but we should keep in mind that the final decision depends on the head count, the available budget, and priorities defined by leaders. 

Lets calculate how much more budget is needed if US army decides to upgrade the meail plan to tasty/problem2:

*Extra cost per person per day =* 0.2

*The active duty U.S. Army personnel =* 479,785

*The estimated number of days personnel are on duty =* 330

**Total Additional Cost =**  0.2 X 479,785 X 330 = 31,665,810  

